# Raptor Mini Colab Deployment

This notebook deploys Raptor Mini (lightweight AI analysis) with ngrok tunneling for the ForgeMonorepo documentation system.

## What this does:
- Installs Raptor Mini dependencies
- Sets up ngrok tunneling
- Provides API endpoints for document analysis
- Makes the service accessible via public URL

## API Endpoints:
- `GET /health` - Health check
- `POST /analyze` - Document analysis

## Usage:
1. Run all cells in order
2. Copy the ngrok URL from the output
3. Update your `.env` file: `RAPTOR_MINI_URL=https://xxxxx.ngrok.io`

In [5]:
# Install dependencies
!pip install fastapi uvicorn requests python-multipart pyngrok
!pip install torch transformers  # For basic AI capabilities
print("✅ Dependencies installed")


✅ Dependencies installed


In [ ]:
# Set up ngrok authentication
# SECURITY: For production, use environment variables or Colab secrets
# You can set this in Colab: Runtime > Change runtime type > Environment variables
import os

# Try to get token from environment variable first (more secure)
NGROK_AUTH_TOKEN = os.getenv(
    "NGROK_AUTH_TOKEN", "36cs1FkRw1Jua3GvHbyU2Smuood_3XfPCs2Jok9MHwANU8G9H"
)

# Fallback to hardcoded token (less secure - only for development)
if not NGROK_AUTH_TOKEN or NGROK_AUTH_TOKEN == "36cs1FkRw1Jua3GvHbyU2Smuood_3XfPCs2Jok9MHwANU8G9H":
    print("❌ ngrok auth token not found!")
    print("   Option 1 (Recommended): Set environment variable NGROK_AUTH_TOKEN")
    print("   Option 2 (Development only): Replace the token below")
    print("   Get token from: https://dashboard.ngrok.com/get-started/your-authtoken")
    NGROK_AUTH_TOKEN = (
        "36cs1FkRw1Jua3GvHbyU2Smuood_3XfPCs2Jok9MHwANU8G9H"  # Development fallback
    )

if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "36cs1FkRw1Jua3GvHbyU2Smuood_3XfPCs2Jok9MHwANU8G9H":
    !ngrok config add-authtoken $NGROK_AUTH_TOKEN
    print("✅ ngrok authenticated")
else:
    print("❌ Please set your ngrok auth token!")
    print("   1. Go to https://dashboard.ngrok.com/get-started/your-authtoken")
    print("   2. Copy your auth token")
    print("   3. Set environment variable: NGROK_AUTH_TOKEN=your_token")
    print("   4. Or replace the token in the code above")
    print("   5. Re-run this cell")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ ngrok authenticated


In [7]:
# Create Raptor Mini API server
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
import threading
import time
import re

app = FastAPI(title="Raptor Mini", description="Lightweight document analysis API")


class AnalysisRequest(BaseModel):
    task: str
    content: str
    analysis_type: str = "quality_score"


@app.get("/health")
async def health_check():
    return {"status": "healthy", "service": "raptor-mini-colab", "version": "1.0.0"}


@app.post("/analyze")
async def analyze_document(request: AnalysisRequest):
    """Basic document analysis using simple heuristics"""
    content = request.content

    # Simple quality scoring
    score = 50  # Base score

    # Length bonus
    if len(content) > 100:
        score += 20
    if len(content) > 500:
        score += 15

    # Structure bonus
    if "##" in content:
        score += 10  # Has sections
    if "```" in content:
        score += 5  # Has code blocks
    if "- " in content or "* " in content:
        score += 5  # Has lists

    # Content quality
    if len(content.split(".")) > 5:
        score += 10  # Multiple sentences

    score = min(100, score)  # Cap at 100

    # Generate feedback
    improvements = []
    if len(content) < 100:
        improvements.append("Add more detailed content")
    if "##" not in content:
        improvements.append("Add section headers for better organization")
    if "```" not in content:
        improvements.append("Include code examples where relevant")

    # Determine strengths and weaknesses
    if score >= 80:
        strength = "Excellent structure and content depth"
        weakness = "Could benefit from more specific examples"
    elif score >= 60:
        strength = "Good basic structure"
        weakness = "Needs more detailed content and examples"
    else:
        strength = "Basic content provided"
        weakness = "Needs significant expansion and structure"

    return {
        "score": score,
        "strength": strength,
        "weakness": weakness,
        "improvements": improvements,
        "analysis_type": request.analysis_type,
    }


def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)


print("🚀 Starting Raptor Mini server...")
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Wait a moment for server to start
time.sleep(2)
print("✅ Server started on port 8000")


🚀 Starting Raptor Mini server...


INFO:     Started server process [892]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


✅ Server started on port 8000


In [8]:
# Start ngrok tunnel
print("🌐 Starting ngrok tunnel...")

# Kill any existing tunnels
ngrok.kill()

# Start new tunnel
public_url = ngrok.connect(8000)
print(f"✅ ngrok tunnel established!")
print(f"🔗 Public URL: {public_url}")
print(f"📋 Copy this URL: {public_url}")

# Most Reliable - Add to Your Raptor Mini Setup
import time
import requests
import json

# Wait for ngrok to start
time.sleep(3)

try:
    # Get the public URL
    tunnels = requests.get("http://localhost:4040/api/tunnels").json()
    ngrok_url = tunnels["tunnels"][0]["public_url"]
    print(f"\n✅ ngrok Public URL: {ngrok_url}")
    print(f"\n📋 Copy this URL: {ngrok_url}")
    # Update public_url to use the API result
    public_url = ngrok_url
except:
    print(
        "⚠️  Could not fetch ngrok URL automatically. Check the output above for the URL."
    )

print(f"\n📝 Update your .env file with:")
print(f"   RAPTOR_MINI_URL={public_url}")
print(f"\n🔄 The tunnel will stay active as long as this notebook runs!")


🌐 Starting ngrok tunnel...


ERROR:pyngrok.process.ngrok:t=2025-12-09T21:51:03+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified is properly formed, but it is invalid.\nYour authtoken: 36cs1FkRw1Jua3GvHbyU2Smuood_3XfPCs2Jok9MHwANU8G9H\nThis usually happens when:\n    - You reset your authtoken\n    - Your authtoken was for a team account that you were removed from\n    - You are using ngrok link and this credential was explicitly revoked\nGo to your ngrok dashboard and double check that your authtoken is correct:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_107\r\n"
ERROR:pyngrok.process.ngrok:t=2025-12-09T21:51:03+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified is properly formed, but it is invalid.\nYour authtoken: 36cs1FkRw1Jua3GvHbyU2Smuood_3XfPCs2Jok9MHwANU8G9H\nThis usually happens when:\n    - You reset your authtoken\n    - Your authtoken was 

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified is properly formed, but it is invalid.\nYour authtoken: 36cs1FkRw1Jua3GvHbyU2Smuood_3XfPCs2Jok9MHwANU8G9H\nThis usually happens when:\n    - You reset your authtoken\n    - Your authtoken was for a team account that you were removed from\n    - You are using ngrok link and this credential was explicitly revoked\nGo to your ngrok dashboard and double check that your authtoken is correct:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_107\r\n.

In [ ]:
# Test the API endpoints
import requests

print("🧪 Testing API endpoints...")

# Test health endpoint
try:
    health_response = requests.get(f"{public_url}/health")
    if health_response.status_code == 200:
        print("✅ Health endpoint working")
        print(f"   Response: {health_response.json()}")
    else:
        print(f"❌ Health endpoint failed: {health_response.status_code}")
except Exception as e:
    print(f"❌ Health test failed: {e}")

# Test analyze endpoint
try:
    test_payload = {
        "task": "analyze_document",
        "content": "This is a test document for quality analysis. It has multiple sentences and some structure.",
        "analysis_type": "quality_score",
    }
    analyze_response = requests.post(f"{public_url}/analyze", json=test_payload)
    if analyze_response.status_code == 200:
        print("✅ Analyze endpoint working")
        result = analyze_response.json()
        print(f"   Score: {result.get('score', 'N/A')}/100")
        print(f"   Strength: {result.get('strength', 'N/A')}")
    else:
        print(f"❌ Analyze endpoint failed: {analyze_response.status_code}")
        print(f"   Response: {analyze_response.text}")
except Exception as e:
    print(f"❌ Analyze test failed: {e}")

print("\n🎉 Raptor Mini is ready!")
print(f"🔗 Your API URL: {public_url}")
print("📝 Update your ForgeMonorepo .env file with this URL")


## 🎯 Next Steps

1. **Copy the ngrok URL** from the output above
2. **Update your `.env` file** in ForgeMonorepo:
   ```
   RAPTOR_MINI_URL=https://xxxxx.ngrok.io
   ```
3. **Test the integration**:
   ```bash
   cd /path/to/ForgeMonorepo
   python3 test_raptor_ready.py
   ```

## 🔄 Keeping it Running

- Keep this Colab notebook open and running
- The ngrok tunnel will stay active
- If the connection drops, re-run the ngrok cell

## 🛠️ Troubleshooting

- If ngrok fails: Check your auth token and internet connection
- If API fails: Check the server logs in the cell output
- If connection drops: Re-run the ngrok tunnel cell